In [2]:
!pip install astroquery
#!pip install streamlit 

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from astroquery.gaia import Gaia

In [8]:


# Function to get star data from Gaia DR3 catalog
def get_star_data():
    # Define the SQL query to retrieve star data
    query = """
    SELECT TOP 1000 source_id, ra, dec, parallax, phot_g_mean_mag
    FROM gaiadr3.gaia_source
    WHERE parallax > 0
    """
    # Execute the query and return the results as a pandas DataFrame
    job = Gaia.launch_job(query)
    return job.get_results().to_pandas()

# Example usage
if __name__ == "__main__":
    star_data = get_star_data()
    print(star_data.head())  # Display the first few rows of the star data


             SOURCE_ID          ra        dec  parallax  phot_g_mean_mag
0  5857770899950692096  201.853993 -65.846630  0.417256        20.619917
1  5857770899950692352  201.855872 -65.846097  1.695592        20.642719
2  5857770899950693248  201.847615 -65.840968  1.684743        20.463293
3  5857770899950694272  201.862971 -65.843786  0.512697        20.010443
4  5857770899950694912  201.856831 -65.839766  1.330915        20.448872


In [5]:
import pandas as pd
import plotly.graph_objects as go
import streamlit as st

# Function to transform coordinates based on the observer's location on the exoplanet
def transform_coordinates(ra, dec, parallax, exoplanet_coords):
    # Simple example of transforming coordinates
    # This should be adjusted for the observer's location on the exoplanet
    # Convert ra and dec from degrees to radians
    ra_rad = ra * (3.14159 / 180)
    dec_rad = dec * (3.14159 / 180)

    # Calculate the distance to the star in parsecs
    distance = 1 / parallax if parallax > 0 else float('inf')

    # Convert to Cartesian coordinates
    x = distance * np.cos(dec_rad) * np.cos(ra_rad)
    y = distance * np.cos(dec_rad) * np.sin(ra_rad)
    z = distance * np.sin(dec_rad)

    # Adjust for the exoplanet coordinates (simple translation)
    transformed_coords = (x - exoplanet_coords[0], y - exoplanet_coords[1], z - exoplanet_coords[2])
    
    return transformed_coords

# Function to plot stars from the perspective of the selected exoplanet
def plot_stars(star_data, exoplanet_coords):
    # Calculate transformed coordinates for each star
    star_data["transformed_coords"] = star_data.apply(
        lambda row: transform_coordinates(row["ra"], row["dec"], row["parallax"], exoplanet_coords),
        axis=1
    )

    # Create 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=star_data["transformed_coords"].apply(lambda x: x[0]),
        y=star_data["transformed_coords"].apply(lambda x: x[1]),
        z=star_data["transformed_coords"].apply(lambda x: x[2]),
        mode='markers',
        marker=dict(size=3, color='blue', opacity=0.8)
    )])
    
    fig.update_layout(scene=dict(
        xaxis_title='X (pc)',
        yaxis_title='Y (pc)',
        zaxis_title='Z (pc)'
    ))
    return fig

# Streamlit app
st.title("Exoplanet Sky Viewer")

# Assume you have some star_data and exoplanet_coords
# star_data should be a DataFrame with columns 'ra', 'dec', 'parallax'
# exoplanet_coords should be a tuple of (x, y, z) coordinates for the exoplanet

# Example star data (this would usually come from your star catalog query)
star_data = pd.DataFrame({
    'ra': [30.0, 60.0],  # Right Ascension in degrees
    'dec': [20.0, -10.0],  # Declination in degrees
    'parallax': [0.1, 0.05]  # Parallax in arcseconds
})

# Example exoplanet coordinates (in parsecs)
exoplanet_coords = (0.5, 0.5, 0.5)

# Visualize the night sky from the exoplanet
st.write("Visualizing the night sky from the exoplanet...")
st.plotly_chart(plot_stars(star_data, exoplanet_coords))


2024-09-23 18:23:29.274 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 18:23:29.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 18:23:29.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 18:23:29.281 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 18:23:29.282 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-23 18:23:29.283 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


NameError: name 'np' is not defined